8. Pessoas que fazem certos tipos de transação fazem mais churn

In [ ]:
from google.colab import auth
import pandas as pd

# get credentials
auth.authenticate_user()
query = '''
WITH transactions_churner AS (SELECT t.transactions_type, t.user_id,u.churner
FROM `e-tensor-411113.neobankclean.transactions_clean` t
LEFT JOIN `e-tensor-411113.neobankclean.userdevice_clean` u USING(user_id))

SELECT transactions_type,
COUNT(transactions_type) AS total_transactions,
COUNT(DISTINCT(user_id)) AS users_per_transaction,
ROUND(SUM(churner)/COUNT(churner),2) AS churner_rate

FROM transactions_churner
GROUP BY transactions_type
HAVING users_per_transaction > 1000
ORDER BY churner_rate DESC
'''

df = pd.read_gbq(query, project_id="neural-tangent-411113")

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   transactions_type      7 non-null      object 
 1   total_transactions     7 non-null      Int64  
 2   users_per_transaction  7 non-null      Int64  
 3   churner_rate           7 non-null      float64
dtypes: Int64(2), float64(1), object(1)
memory usage: 366.0+ bytes


In [ ]:
df

,transactions_type,total_transactions,users_per_transaction,churner_rate
0,FEE,20285,8929,0.07
1,TOPUP,196863,16342,0.05
2,ATM,53521,7727,0.04
3,CARD_PAYMENT,1190350,13720,0.02
4,EXCHANGE,94521,6990,0.02
5,TRANSFER,342725,10217,0.01
6,CARD_REFUND,8765,3423,0.01


In [ ]:
round(df['churner_rate'].describe(),3)

count    7.000
mean     0.031
std      0.023
min      0.010
25%      0.015
50%      0.020
75%      0.045
max      0.070
Name: churner_rate, dtype: float64

In [ ]:
import plotly.express as px

fig = px.bar(df, x='transactions_type', y='churner_rate',
             title='Taxa de churn por tipo de transação')
fig.update_layout(xaxis_title='', yaxis_title='')
fig.show()

HO: Não há relação entre o tipo de transação e o churn rate

H1: Há relação entre o tipo de transação e o churn rate

**CONCLUSÃO:** há variação na taxa de churn, porém ela é pouco significativa, sugerindo a rejeição da hipótese alternativa.